<a href="https://colab.research.google.com/github/YoheiFukuhara/nlp100-2020/blob/main/06.%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92/51_%E7%89%B9%E5%BE%B4%E9%87%8F%E6%8A%BD%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

import pandas as pd
import spacy
#from nltk.stem.porter import PorterStemmer as PS
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python --version
!pip show google pandas spacy

Python 3.7.12
Name: google
Version: 2.0.3
Summary: Python bindings to the Google search engine.
Home-page: http://breakingcode.wordpress.com/
Author: Mario Vilas
Author-email: mvilas@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: beautifulsoup4
Required-by: 
---
Name: pandas
Version: 1.1.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: None
Author-email: None
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: pytz, python-dateutil, numpy
Required-by: xarray, vega-datasets, statsmodels, sklearn-pandas, seaborn, pymc3, plotnine, pandas-profiling, pandas-gbq, pandas-datareader, mlxtend, mizani, holoviews, gspread-dataframe, google-colab, fix-yahoo-finance, fbprophet, fastai, cufflinks, cmdstanpy, arviz, altair
---
Name: spacy
Version: 2.2.4
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Expl

In [ ]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/06.MachineLearning/'
nlp = spacy.load('en_core_web_sm')

reg_sym = re.compile(r'^[!-/:-@[-`{-~]|[!-/:-@[-`{-~]$')
reg_dit = re.compile('[0-9]')

# Stemming をすると学習済ベクトルでのEmbeddingができなくなるので中止
#ps = PS()

# RNN・CNN用なので、そんなに除去しない
EXC_POS = {'PUNCT',   # 句読点
           'X',       # その他
           'SYM',     # 記号
           'SPACE' }   # SPACE
 #          'PART',    # 助詞('sなど)
 #          'CCONJ',   # 接続詞(andなど)
 #          'AUX',     # 助動詞(wouldなど)
 #          'PRON',    # 代名詞
 #          'SCONJ',   # 従位接続詞(whetherなど)
#           'ADP',     # 接置詞(inなど)
#           'NUM'}     # 番号

In [ ]:
def is_stopword(token):
    lemma = reg_sym.sub('', token.lemma_)
    return True if lemma == '' \
                 or token.pos_ in EXC_POS \
                 or len(lemma) == 1 \
                 or reg_dit.search(lemma) \
                 else False
# token.is_stop  ストップワード除外しない

def preproc_text(text):
    doc = nlp(text)
#    return ' '.join([ps.stem(reg_sym.sub('', token.lemma_)) for token in doc if is_stopword(token) is False])
    return ' '.join([reg_sym.sub('', token.lemma_) for token in doc if is_stopword(token) is False])
    
def change_text(type_):
    df = pd.read_table(BASE_PATH+type_+'.txt')
    print('---'+type_+'---')
    df.info()
    df['title'] = df['title'].map(preproc_text)
    df.to_csv(BASE_PATH+type_+'.feature.txt', sep='\t', index=False)

In [ ]:
%%time
change_text('train')
change_text('valid')
change_text('test')

---train---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10684 entries, 0 to 10683
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     10684 non-null  object
 1   category  10684 non-null  object
dtypes: object(2)
memory usage: 167.1+ KB
---valid---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     1336 non-null   object
 1   category  1336 non-null   object
dtypes: object(2)
memory usage: 21.0+ KB
---test---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     1336 non-null   object
 1   category  1336 non-null   object
dtypes: object(2)
memory usage: 21.0+ KB
CPU times: user 2min 1s, sys: 391 ms, total: 2min 1s
Wall time: 2

In [ ]:
# More Than $3.2 Billion?
# What's in a name?
def check_stop_word(sentence):
    doc = nlp(sentence)
    for token in doc:
        lemma = reg_sym.sub('', token.lemma_)
        if lemma == '' \
                 or token.pos_ in EXC_POS \
                 or len(lemma) == 1 \
                 or reg_dit.search(lemma):
            print('stop word!!!')
        print(f'lemma: {lemma}')
        if token.pos_ in EXC_POS:
            print(f'pos: {token.pos_}')
        print(f': {len(lemma)}')
        print(f': {reg_dit.search(lemma)}')
        print('-----')

check_stop_word("More Than $3.2 Billion?")
check_stop_word("What's in a name?")

lemma: More
: 4
: None
-----
lemma: than
: 4
: None
-----
stop word!!!
lemma: 
pos: SYM
: 0
: None
-----
stop word!!!
lemma: 3.2
: 3
: <re.Match object; span=(0, 1), match='3'>
-----
lemma: billion
: 7
: None
-----
stop word!!!
lemma: 
pos: PUNCT
: 0
: None
-----
lemma: what
: 4
: None
-----
lemma: be
: 2
: None
-----
lemma: in
: 2
: None
-----
stop word!!!
lemma: a
: 1
: None
-----
lemma: name
: 4
: None
-----
stop word!!!
lemma: 
pos: PUNCT
: 0
: None
-----
